# Installation

In [61]:
!pip install openrouteservice

In [62]:
import geopandas as gpd
from shapely.geometry import Point
import folium
import openrouteservice
from pprint import pprint
import requests
import pandas as pd
import numpy as np
import time
from pprint import pprint
from tqdm import tqdm
from google.colab import drive
from geopy.distance import geodesic
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lecture

In [63]:
# URL du fichier GeoJSON (assurez-vous que l'URL est correcte)
url = "https://opendata.paris.fr/explore/dataset/perimetresqaa/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

# Charger les données GeoJSON dans un GeoDataFrame
gdf = gpd.read_file(url)

# Afficher les premières lignes du GeoDataFrame pour vérifier le chargement des données
print(gdf.head().to_markdown())

|    |   sect_adm |   st_perimeter_shape | geometry                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

# Localisation d'un point dans un des sites des JO

In [64]:
# Exemple de coordonnées du point à vérifier
latitude = 48.85320333578678
longitude = 2.3358160699135233

# Créer un objet Point
point = Point(longitude, latitude)

# Vérifier si le point est dans une des zones
is_in_zone = gdf.contains(point)

# Afficher le résultat
print(gdf[is_in_zone].to_markdown())

|    |   sect_adm |   st_perimeter_shape | geometry                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [65]:
latitude = 48.85320333578678
longitude = 2.3358160699135233

# Créer une carte centrée sur les coordonnées du point
m1 = folium.Map(location=[latitude, longitude],           # Position du point
                zoom_start=15,                            # Niveau du zoom au début
                tiles="Cartodb Positron",                 # Définit le fond : "Cartodb Positron"  blanc / "Cartodb dark_matter" : noir / "OpenStreetMap" (défaut) maps)
                width='80%', height='80%')                # Gère la taille de la map

# Ajouter un marqueur pour le point
folium.Marker(
    location=[latitude, longitude],  # Coordonnées du point
    popup='Point d\'intérêt',        # Texte de la popup au clic
    icon=folium.Icon(color='blue')   # Couleur de l'icône du marqueur
).add_to(m1)

# Enregistrer la carte dans un fichier HTML
m1.save('map_with_point.html')

# Afficher la carte (uniquement si vous êtes dans un environnement Jupyter)
m1

# Localisation des sites des JO

In [66]:
# URL du fichier GeoJSON (assurez-vous que l'URL est correcte)
url = "https://opendata.paris.fr/explore/dataset/perimetresqaa/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

# Charger les données GeoJSON dans un GeoDataFrame
gdf = gpd.read_file(url)

# Créer une carte centrée sur Paris
center = [48.85794, 2.33312]  # Vous pouvez ajuster les coordonnées si nécessaire
m2 = folium.Map(location=center,
                zoom_start=13,
                tiles = "Cartodb Positron")

# Ajouter les données GeoDataFrame à la carte
folium.GeoJson(gdf,
               style_function=lambda feature: {
                                               "fillColor": "red",              # Couleur de remplissage (en nom ou en hexa)
                                               "fillOpacity": 0.3,              # Opacité du remplissage (0 à 1 - peut dépasser mais ne changera rien)
                                               "color": "red",                # Couleur des bordures (en nom ou en hexa)
                                               "weight": 2,                     # Epaisseur du trait
                                               "dashArray": "10, 5, 2, 5"              # Permet de mettre en Pointillé (opt) nombre de pixel plein, nombre de pixel vide
                                               }
               ).add_to(m2)

# Exemple de coordonnées du point à vérifier
latitude = 48.85320333578678
longitude = 2.3358160699135233

# Créer un marqueur pour le point
folium.Marker(
    location=[latitude, longitude],                                                                                   # Coordonnées du point
    popup='Point d\'intérêt',                                                                                         # Texte de la popup au clic
    # icon=folium.Icon(icon = cloud, color='blue')                                                                    # Gestion d'une icone classique (forme + couleur)
    icon=folium.CustomIcon('https://storage.googleapis.com/endurance-apps-liip/media/cache/olympics-games_asset_card_grid_fs/622093e093e86c66c21cfc9b', icon_size=(30, 30))         # Gestion d'une icone personnalisé (lien de l'image + taille souhaité)
).add_to(m2)


# Enregistrer la carte dans un fichier HTML
m2.save('paris_qaa_map_with_point.html')

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m2


# Test d'une recherche de trajet avec itinéraire

In [67]:
# Remplacez par votre propre clé API OpenRouteService
api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Points de départ et d'arrivée
start_coords = (2.3358160699135233, 48.85320333578678)  # Coordonnées du point de départ (longitude, latitude)
end_coords = (2.2943506, 48.8588443)  # Coordonnées du point d'arrivée (longitude, latitude)

# Obtenir un itinéraire
route = client.directions(
    coordinates=[start_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

# Afficher la réponse brute
pprint(route)

{'bbox': [2.293821, 48.853191, 2.335809, 48.859966],
 'features': [{'bbox': [2.293821, 48.853191, 2.335809, 48.859966],
               'geometry': {'coordinates': [[2.335809, 48.853191],
                                            [2.335787, 48.853197],
                                            [2.335593, 48.853247],
                                            [2.335653, 48.853301],
                                            [2.335666, 48.853313],
                                            [2.335611, 48.853332],
                                            [2.335582, 48.853339],
                                            [2.335551, 48.853347],
                                            [2.335448, 48.853385],
                                            [2.335432, 48.85339],
                                            [2.335421, 48.853401],
                                            [2.335405, 48.853411],
                                            [2.335386, 48.853417],
          

# Localisation d'un lieu à partir de son adresse

In [68]:
# Créer un client OpenRouteService en spécifiant votre clé API
api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Adresse à géolocaliser
adresse = "44 Rue Alphonse Penaud, 75020 Paris"          # Ce format est obligatoire (risque de ne pas trouver le point)

# Géolocalisation de l'adresse
response = client.pelias_search(adresse)

# Extrait des coordonnées géographiques si une réponse est disponible
if response['features']:
    location = response['features'][0]['geometry']['coordinates']
    latitude, longitude = location[1], location[0]
    print("Latitude:", latitude)
    print("Longitude:", longitude)
else:
    print("Adresse introuvable.")

# Créer une carte centrée sur Paris
center = [48.85794, 2.33312]  # Vous pouvez ajuster les coordonnées si nécessaire
m3 = folium.Map(location=center,
                zoom_start=13,
                tiles = "Cartodb Positron")

folium.Marker(
    location=[latitude, longitude],
    popup= folium.Popup(f'Wild Code School - {adresse}',        # Texte a afficher (ici lieu + adresse)
                        max_width=300, min_width=200),          # Choisir la taille max du cadre
    icon=folium.Icon(icon = "cloud", color='blue')
).add_to(m3)

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m3

Latitude: 48.868323
Longitude: 2.406794


#Récupérer la longitude et latitude

In [69]:
import geopandas as gpd
from shapely.geometry import Point
import folium
import openrouteservice
from pprint import pprint

# Créer un client OpenRouteService en spécifiant votre clé API
api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Adresse à géolocaliser
adresse = "44 Rue Alphonse Penaud, 75020 Paris"          # Ce format est obligatoire (risque de ne pas trouver le point)

# Géolocalisation de l'adresse
response = client.pelias_search(adresse)

# Extrait des coordonnées géographiques si une réponse est disponible
if response['features']:
    location = response['features'][0]['geometry']['coordinates']
    latitude, longitude = location[1], location[0]
    print("Latitude:", latitude)
    print("Longitude:", longitude)
else:
    print("Adresse introuvable.")

Latitude: 48.868323
Longitude: 2.406794


# Création d'un itinéraire

In [70]:
# Remplacez par votre propre clé API OpenRouteService

api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Points de départ et d'arrivée
start_coords = (2.3358160699135233, 48.85320333578678)  # Coordonnées du point de départ (longitude, latitude)
end_coords = (2.2943506, 48.8588443)  # Coordonnées du point d'arrivée (longitude, latitude)

# Obtenir un itinéraire
route = client.directions(  coordinates=[start_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

# Afficher la réponse brute pour vérification (facultatif)
#print(route)

# Extraire les coordonnées de l'itinéraire
route_coords = route['features'][0]['geometry']['coordinates']
route_coords = [(coord[1], coord[0]) for coord in route_coords]  # Inverser latitude et longitude

# Créer une carte centrée sur le point de départ
m4 = folium.Map(location=[start_coords[1], start_coords[0]], zoom_start=14, tiles ="Cartodb Positron")

# Ajouter un marqueur pour le point de départ
folium.Marker(
    location=[start_coords[1], start_coords[0]],
    popup='Départ',
    icon=folium.Icon(color='green')
).add_to(m4)


# Ajouter un marqueur pour le point d'arrivée
folium.Marker(
    location=[end_coords[1], end_coords[0]],
    popup='Arrivée',
    icon=folium.Icon(color='red')
).add_to(m4)

# Ajouter l'itinéraire à la carte
folium.PolyLine(
    route_coords,
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m4)

# Enregistrer la carte dans un fichier HTML
m4.save('paris_itineraire.html')

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m4


#Calcul de temps de trajet

In [71]:
# Remplacez par votre propre clé API OpenRouteService
api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Points de départ et d'arrivée
start_coords = (2.3358160699135233, 48.85320333578678)  # Coordonnées du point de départ (longitude, latitude)
end_coords = (2.2943506, 48.8588443)  # Coordonnées du point d'arrivée (longitude, latitude)

# Modes de transport à tester
modes_transport = ['driving-car', 'foot-walking', 'cycling-regular']

# Récupérer le temps de trajet pour chaque mode de transport
for mode in modes_transport:
    # Obtenir un itinéraire
    route = client.directions(
        coordinates=[start_coords, end_coords],
        profile=mode,
        format='geojson'
    )

    # Extraire le temps de trajet
    duration = route['features'][0]['properties']['segments'][0]['duration']

    print(f"Temps de trajet en {mode}: {duration} secondes")


"""
Avec l'API OpenRouteService, vous pouvez spécifier différents profils de transport pour obtenir des itinéraires adaptés
à différents types de déplacements. Voici une liste des profils de transport disponibles :

--------- Voiture ---------

'driving-car': Conduite en voiture (par défaut).
'driving-hgv': Conduite en poids lourds (camions).

--------- Vélo ---------

'cycling-regular': Vélo de ville (par défaut).
'cycling-road': Vélo de route.
'cycling-mountain': VTT.
'cycling-electric': Vélo électrique.

--------- Marche ---------

'foot-walking': Marche à pied (par défaut).

--------- Transport en commun ---------

'driving-car': Transports en commun (itinéraires incluant les transports en commun).

--------- Route à cheval ---------

'horse-riding': Itinéraires à cheval.

--------- Navire ---------

'sailing-boat': Navigation en bateau.

--------- Handicap ---------

'wheelchair': Accessibilité en fauteuil roulant.
"""

Temps de trajet en driving-car: 700.9 secondes
Temps de trajet en foot-walking: 2544.8 secondes
Temps de trajet en cycling-regular: 820.4 secondes


"\nAvec l'API OpenRouteService, vous pouvez spécifier différents profils de transport pour obtenir des itinéraires adaptés\nà différents types de déplacements. Voici une liste des profils de transport disponibles :\n\n--------- Voiture ---------\n\n'driving-car': Conduite en voiture (par défaut).\n'driving-hgv': Conduite en poids lourds (camions).\n\n--------- Vélo ---------\n\n'cycling-regular': Vélo de ville (par défaut).\n'cycling-road': Vélo de route.\n'cycling-mountain': VTT.\n'cycling-electric': Vélo électrique.\n\n--------- Marche ---------\n\n'foot-walking': Marche à pied (par défaut).\n\n--------- Transport en commun ---------\n\n'driving-car': Transports en commun (itinéraires incluant les transports en commun).\n\n--------- Route à cheval ---------\n\n'horse-riding': Itinéraires à cheval.\n\n--------- Navire ---------\n\n'sailing-boat': Navigation en bateau.\n\n--------- Handicap ---------\n\n'wheelchair': Accessibilité en fauteuil roulant.\n"

#Trajectoire de la flamme (prévision)

In [72]:
# Remplacez par votre propre clé API OpenRouteService

api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Points de départ et d'arrivée

# Point d'avant Paris
start_coords = (2.454307999480008,48.83374390337135)


middle_coords = (2.362897133418489,48.85706328709478)


# Point d'après Paris
end_coords = (2.2533449626886175,48.84546466556765)



# Obtenir un itinéraire
route2 = client.directions( coordinates=[middle_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

route1 = client.directions( coordinates=[start_coords, middle_coords],
    profile='foot-walking',
    format='geojson'
)



# Afficher la réponse brute pour vérification (facultatif)
#print(route)

# Extraire les coordonnées de l'itinéraire
route_coords = route1['features'][0]['geometry']['coordinates']
route_coords = [(coord[1], coord[0]) for coord in route_coords]  # Inverser latitude et longitude


# Extraire les coordonnées de l'itinéraire
route_coords2 = route2['features'][0]['geometry']['coordinates']
route_coords2 = [(coord[1], coord[0]) for coord in route_coords2]  # Inverser latitude et longitude


# Créer une carte centrée sur le point de départ
m4 = folium.Map(location=[start_coords[1], start_coords[0]], zoom_start=14, tiles ="Cartodb Positron")

# Ajouter un marqueur pour le point des lacs de la forêt d'Orient
folium.Marker(
    location=[start_coords[1], start_coords[0]],
    popup='INSEP',
    icon=folium.Icon(color='green')
).add_to(m4)


# Ajouter un marqueur pour le deuxième point
folium.Marker(
    location=[middle_coords[1], middle_coords[0]],
    popup='Musée Carnavalet',
    icon=folium.Icon(color='bleu')
).add_to(m4)


# Ajouter un marqueur pour le point d'arrivée
folium.Marker(
    location=[end_coords[1], end_coords[0]],
    popup='Court Simonne-Mathieu',
    icon=folium.CustomIcon('https://storage.googleapis.com/endurance-apps-liip/media/cache/olympics-games_asset_card_grid_fs/622093e093e86c66c21cfc9b',
                           icon_size=(50, 50))
).add_to(m4)

# Ajouter l'itinéraire à la carte
folium.PolyLine(
    route_coords,
    color='yellow',
    weight=5,
    opacity=0.7
).add_to(m4)

folium.PolyLine(
    route_coords2,
    color='yellow',
    weight=5,
    opacity=0.7
).add_to(m4)

# Enregistrer la carte dans un fichier HTML
m4.save('paris_itineraire.html')

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m4


<ipython-input-72-3cd4b62ed89e>:61: UserWarning: color argument of Icon should be one of: {'darkblue', 'darkgreen', 'darkpurple', 'lightblue', 'darkred', 'purple', 'white', 'black', 'green', 'pink', 'cadetblue', 'lightred', 'lightgray', 'beige', 'red', 'blue', 'gray', 'orange', 'lightgreen'}.
  icon=folium.Icon(color='bleu')


# Code modèle pour calcul d'itinéraire

In [73]:
# Remplacez par votre propre clé API OpenRouteService

api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Points de départ et d'arrivée
# Mettre les coordonnées de l'adresse de la personne
start_coords = (2.3358160699135233, 48.85320333578678)  # Coordonnées du point de départ (longitude, latitude)

#Mettre les coordonnées de l'adresse du restaurant
end_coords = (2.2943506, 48.8588443)  # Coordonnées du point d'arrivée (longitude, latitude)

# Obtenir un itinéraire
route = client.directions(  coordinates=[start_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

# Afficher la réponse brute pour vérification (facultatif)
#print(route)

# Extraire les coordonnées de l'itinéraire
route_coords = route['features'][0]['geometry']['coordinates']
route_coords = [(coord[1], coord[0]) for coord in route_coords]  # Inverser latitude et longitude

# Créer une carte centrée sur le point de départ
m4 = folium.Map(location=[start_coords[1], start_coords[0]], zoom_start=14, tiles ="Cartodb Positron")

# Ajouter un marqueur pour le point de départ
folium.Marker(
    location=[start_coords[1], start_coords[0]],
    popup='Départ',
    icon=folium.Icon(color='green')
).add_to(m4)


# Ajouter un marqueur pour le point d'arrivée
folium.Marker(
    location=[end_coords[1], end_coords[0]],
    popup='Arrivée',
    icon=folium.CustomIcon('https://storage.googleapis.com/endurance-apps-liip/media/cache/olympics-games_asset_card_grid_fs/622093e093e86c66c21cfc9b', # Remplacer le lien par celui de l'icone (fonctionne avec les fichiers locaux)
                           icon_size=(50, 50))
).add_to(m4)

# Ajouter l'itinéraire à la carte
folium.PolyLine(
    route_coords,
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m4)

# Enregistrer la carte dans un fichier HTML
m4.save('paris_itineraire.html')  #Voir pour remplacer le nom avec le nom du restaurant

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m4


# Etablir un rayon autour d'un point

In [74]:
import folium

# Coordonnées du centre du cercle (latitude, longitude)
center_coordinates = (48.8566, 2.3522)  # Exemple : Paris

# Créez une carte centrée sur ces coordonnées
map = folium.Map(location=center_coordinates, zoom_start=13)

# Ajoutez un cercle avec un rayon (en mètres)
radius = 1000  # Exemple : 1000 mètres
circle = folium.Circle(
    location=center_coordinates,
    radius=radius,
    color='blue',
    fill=True,
    fill_color='blue'
)

# Ajoutez le cercle à la carte
circle.add_to(map)

# Enregistrez la carte dans un fichier HTML
map.save('map_with_circle.html')

map

In [76]:
# Coordonnées du centre du cercle (latitude, longitude)
# Représente l'adresse de la personne
center_coordinates = (48.86449539923668, 2.392065890514932)   # Exemple : Paris

# Rayon du cercle en mètres
radius = 1000  # Exemple : 1000 mètres

# Coordonnées du point à vérifier (latitude, longitude)
# Adresse des restaurant
point_coordinates = (48.8645459983239, 2.397434149895346)  # Exemple : Louvre

# Calculer la distance entre le centre et le point
distance = geodesic(center_coordinates, point_coordinates).meters

# Vérifier si la distance est inférieure ou égale au rayon
if distance <= radius:
    print("Le point est à l'intérieur du cercle.")
else:
    print("Le point est à l'extérieur du cercle.")

Le point est à l'intérieur du cercle.


# Partie utilisable - Vérifier la présence des resto dans le périmètre

## Import du DF tripadvisor

In [75]:
# Resto_trip_advisor
df = pd.read_csv("/content/drive/MyDrive/PARIS 0224 - Data Analyst/Datathon/Restolympic/Datasets/Dataset - Pré nettoyage/Copie de tripadvisor_restos.csv",sep=";")

In [85]:
print(df.head().to_markdown())

|    | restaurant_link   | restaurant_name   | city   | address                                                 |   latitude |   longitude | top_tags                                 | price_level   | meals                                  | cuisines                   |   special_diets | features                                                                                                                                                                                   | vegetarian_friendly   | vegan_options   | gluten_free   | original_open_hours                                                                                                                                                      |   avg_rating |   total_reviews_count |   excellent |   very_good |   average |   poor |   terrible |   food |   service |   value |
|---:|:------------------|:------------------|:-------|:--------------------------------------------------------|-----------:|------------:|:---------------------

In [80]:
# gestion de la ligne sans pos
df = df.dropna(subset = ["latitude","longitude"])

In [84]:
# on règle l'horaire du resto
df["original_open_hours"] = df["original_open_hours"].fillna("Horaire non connue")

## Récupération des restaurants proches du client

### On récupère la pos du client à partir de son adresse

In [111]:
# Créer un client OpenRouteService en spécifiant votre clé API
api_key = '5b3ce3597851110001cf6248302d9fcb8b44439aa282335994a41406'
client = openrouteservice.Client(key=api_key)

# Adresse à géolocaliser // donné par le client
adresse = "44 Rue Alphonse Penaud, 75020 Paris"          # Ce format est obligatoire (risque de ne pas trouver le point)

# Géolocalisation de l'adresse
response = client.pelias_search(adresse)

# Extrait des coordonnées géographiques si une réponse est disponible
if response['features']:
    location = response['features'][0]['geometry']['coordinates']
    latitude_client, longitude_client = location[1], location[0]
    print("Latitude:", latitude_client)
    print("Longitude:", longitude_client)
else:
    print("Adresse introuvable.")

Latitude: 48.868323
Longitude: 2.406794


### Utilisation de la pos du client pour filtrer les restaurants proches

In [112]:
adresse_client = (latitude_client, longitude_client)   # Récupéré au dessus
radius = 1000  # Rayon du cercle en mètres

In [79]:
# Fonction pour récupérer les restaurants proches

def mes_restos (resto) :
  resto_liste = []
  list_nom_colone = df.columns

  # Boucle qui parcours le dataframe et récupère les pos des restaurants
  for index, row in resto.iterrows():
    distance = geodesic(adresse_client, (float(row["latitude"]), float(row["longitude"]))).meters

    # Si le restaurant est dans le rayon, on le stock dans une liste
    if distance <= radius :
        resto_liste.append(row)

  # On recréer un dataframe contenant uniquement les restaurants proches
  return pd.DataFrame(resto_liste,columns=list_nom_colone)

In [81]:
# On peut utiliser resto_proche maitenant
resto_proche = mes_restos(df)

###On montre les restaurants autour de la personne

In [86]:
print(resto_proche.head().to_markdown())

|     | restaurant_link   | restaurant_name      | city   | address                                          |   latitude |   longitude | top_tags                           | price_level   | meals                                            | cuisines                |   special_diets | features                                                                                                                                               | vegetarian_friendly   | vegan_options   | gluten_free   | original_open_hours                                                                                                                                                                                                                                                               |   avg_rating |   total_reviews_count |   excellent |   very_good |   average |   poor |   terrible |   food |   service |   value |
|----:|:------------------|:---------------------|:-------|:-----------------------------------

In [104]:
# Créer une carte centrée sur le client

m_resto = folium.Map(location=adresse_client,
                zoom_start=13,
                tiles = "Cartodb Positron")

# Marker client
folium.Marker(
    location= adresse_client,
    popup= folium.Popup(f'Vous - {adresse}',
                        max_width=300, min_width=200),
    icon=folium.CustomIcon('https://cdn-icons-png.flaticon.com/512/3537/3537838.png',
                           icon_size=(30, 30))
).add_to(m_resto)

# Boucle pour afficher les différents restaurants

for index, row in resto_proche.iterrows():
    folium.Marker(
                  location= (float(row["latitude"]),float(row["longitude"])),
                  popup= folium.Popup(row["restaurant_name"],
                                      max_width=300, min_width=200),
                  icon=folium.CustomIcon('https://cdn-icons-png.flaticon.com/512/3448/3448609.png',
                           icon_size=(30, 30))
                  ).add_to(m_resto)



# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m_resto

## Calcul du trajet pour un restaurant /!\ longitude/latitude inversé



In [106]:
#Correspond au restaurant choisi
restaurant = resto_proche.head(1)

In [114]:
# Points de départ et d'arrivée
start_coords = (longitude_client,latitude_client)  # Coordonnées du point de départ (longitude, latitude)

#Mettre les coordonnées de l'adresse du restaurant
end_coords = (float(restaurant["longitude"]), float(restaurant["latitude"]))  # Coordonnées du point d'arrivée (longitude, latitude)

# Obtenir un itinéraire
route = client.directions(  coordinates=[start_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

# Afficher la réponse brute pour vérification (facultatif)
#print(route)

# Extraire les coordonnées de l'itinéraire
route_coords = route['features'][0]['geometry']['coordinates']
route_coords = [(coord[1], coord[0]) for coord in route_coords]  # Inverser latitude et longitude

# Créer une carte centrée sur le point de départ
m_trajet = folium.Map(location=[start_coords[1], start_coords[0]], zoom_start=14, tiles ="Cartodb Positron")

# Ajouter un marqueur pour le point de départ
folium.Marker(
    location= adresse_client,
    popup= folium.Popup('Vous'),
    icon=folium.CustomIcon('https://cdn-icons-png.flaticon.com/512/3537/3537838.png',
                           icon_size=(30, 30))
).add_to(m_trajet)

# Ajouter un marqueur pour le point d'arrivée
folium.Marker(
    location=[end_coords[1], end_coords[0]],
    popup=folium.Popup(restaurant["restaurant_name"]),
    icon=folium.CustomIcon('https://cdn-icons-png.flaticon.com/512/3448/3448609.png', # Remplacer le lien par celui de l'icone (fonctionne avec les fichiers locaux)
                           icon_size=(30, 30))
).add_to(m_trajet)

# Ajouter l'itinéraire à la carte
folium.PolyLine(
    route_coords,
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m_trajet)

# Enregistrer la carte dans un fichier HTML
m_trajet.save(f'{restaurant["restaurant_name"]}.html')  #Voir pour remplacer le nom avec le nom du restaurant

# Afficher un aperçu de la carte (uniquement si vous êtes dans un environnement Jupyter)
m_trajet

<ipython-input-114-3354f64ad84a>:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  end_coords = (float(restaurant["longitude"]), float(restaurant["latitude"]))  # Coordonnées du point d'arrivée (longitude, latitude)
